# Experiment 2: influence of an experimental protocol on the immune state prediction

In [2]:
from util.implanting import make_immune_signal, make_exp_protocol_signal

# make immune state signal:

immune_state_signal = make_immune_signal()

# make experimental protocol signal (two signals for two experimental protocols):

exp_protocol1_signal = make_exp_protocol_signal(protocol_id=1)
exp_protocol2_signal = make_exp_protocol_signal(protocol_id=2)


In [3]:
import os
import dagsim.baseDS as ds
import numpy as np
import uuid
from pathlib import Path
from util.load_repertoire import load_olga_repertoire, load_iml_repertoire
from util.implanting import make_immune_signal, make_repertoire_with_signal
from immuneML.util.PathBuilder import PathBuilder
from util.implanting import make_repertoire_without_signal
from util.implanting import make_AIRR_dataset

# os.chdir("./simData")
result_path = PathBuilder.build("./data/implanted/")

In [17]:
def get_state():
    return np.random.binomial(n=1, p=0.5)

def get_protocol(state):
    return np.random.binomial(n=1, p=0.5) if state==1 else np.random.binomial(n=1, p=0.2)

def generate_repertoire(state, protocol, seed):
    uid = str(uuid.uuid4())
    # path = "healthy" if state==1 else "disease"

    PathBuilder.build('./data/olga/')

    os.system("olga-generate_sequences --humanTRB -n 1000 -o ./data/olga/" + uid + ".tsv --seed=" + str(seed))

    rep = load_olga_repertoire(filepath=Path("./data/olga/" + uid + ".tsv"), result_path=Path("./data/Exp2a/immuneML_format/"))

    if state == 0:
        rep = make_repertoire_without_signal(repertoire=rep, signal_name=immune_state_signal.id, path=result_path)
    else:
        rep = make_repertoire_with_signal(rep, immune_state_signal, result_path)

    if protocol == 0:
        make_repertoire_with_signal(rep, exp_protocol1_signal, result_path)
    else:
        make_repertoire_with_signal(rep, exp_protocol2_signal, result_path)

    return uid

In [9]:
Index = ds.Generic(name="index", function=np.arange, size_field="stop")
ImmuneState = ds.Generic(name="state", function=get_state)
Protocol = ds.Generic(name="protocol", function=get_protocol, arguments={"state": ImmuneState})
Repertoire = ds.Generic(name="repertoire", function=generate_repertoire, arguments={"state": ImmuneState, "protocol": Protocol, "seed": Index})

In [ ]:
CAIRR_graph = ds.Graph(name="CAIRR_graph_2a", list_nodes=[Index, Protocol, ImmuneState, Repertoire])
CAIRR_data = CAIRR_graph.simulate(num_samples=10, csv_name="CAIRR_data_2a")


In [10]:
dataset = make_AIRR_dataset(repertoires=[load_iml_repertoire(filepath=Path("./data/Exp2a/immuneML_format/repertoires/"), identifier=id) for id in CAIRR_graph_a["repertoire"]],
                            path=Path("./data/exported/"),
                            dataset_name="Exp2a", signal_names=[immune_state_signal.id])

print(f"Dataset id: {dataset.identifier}, dataset name: {dataset.name}, number of repertoires: {dataset.get_example_count()}")

Starting sequence generation... 
Completed generating all 1000 sequences in 0.07 seconds.


AttributeError: 'str' object has no attribute 'implant_to_repertoire'